In [24]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor
import pandas as pd
import numpy as mp
from matplotlib import pyplot as plt
from textblob import TextBlob
import re
import json

In [2]:
Access_Token = ''
Access_Token_Secret  = ' '
Consumer_Key = ''
Consumer_Secret = ''

In [3]:
class Twitter_Client():
    """
    Twitter Client
    """
    
    def __init__(self, twitter_user=None):
        self.auth = Twitter_Authenticator().Twitter_Authentication_app()
        self.twitter_client = API(self.auth)
        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id = self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return(tweets)

In [4]:
class Twitter_Authenticator():
    """
    Twitter Authentication
    """

    def __init__(self):
        pass

    def Twitter_Authentication_app(self):
        auth  = OAuthHandler(Consumer_Key, Consumer_Secret)
        auth.set_access_token(Access_Token, Access_Token_Secret)
        return(auth)

In [ ]:
class Twitterstreamer():


    def __init__(self):
        self.auth = Twitter_Authenticator()

    def stream_tweets(self, fetched_tweet_filename, hash_tag_list):
        #Handles Twitter authentication and the connection to the Twitter Streaming API
        listener  = TwitterListener()
        auth = self.auth.Twitter_Authentication_app()

        stream = Stream(authlistener)
        stream.filter(track=hash_tag_list)

In [ ]:
class TwitterListener(StreamListener):
    """
    Prints data
    """

    def __init__(self):
        self.fetched_tweet_filename = fetched_tweet_filename

    def on_data(self, data):
        try:
            with open(self.fetched_tweet_filename, 'a') as f:
                f.write(data)
            return True
        except BaseException as e:
            print('Error on data: %s' % str(e))
            return True

    def on_error(selfstatus):
        if status == 420:
            #Returning False on_data method in case rate limit occurs
            return False
        print(status)


In [ ]:
class Tweet_Analyzer:
    """
    Analyzing and Categorizing content from tweets
    """
    
    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," " ,tweet).split())

    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))

        if analysis.sentiment.polarity > 0:
            return 1
        elif analysis.sentiment.polarity == 0:
            return 0
        else:
            return -1

    def tweets_to_dataframe(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns = ['Tweets'])
        df['ID'] = [tweet.id for tweet in tweets]
        df['len'] = [len(tweet.text) for tweet in tweets]
        df['date'] = [tweet.created_at for tweet in tweets]
        df['source'] = [tweet.source for tweet in tweets]
        df['likes'] = [tweet.favorite_count for tweet in tweets]
        df['retweets'] = [tweet.retweet_count for tweet in tweets]
        return df

In [ ]:
if __name__ ==  '__main__':
    hash_tag_list = ['donald trump' 'trade' 'etf']
    fetched_tweet_filename = "tweets.json"

    tweet_analyzer = Tweet_Analyzer()
    twitter_client = Twitter_Client()

    api =   twitter_client.get_twitter_client_api()
    tweets = api.user_timeline(screen_name = "realDonaldTrump", count = 200)
    df = tweet_analyzer.tweets_to_dataframe(tweets)
    df['Sentiment'] = [tweet_analyzer.analyze_sentiment(tweet) for tweet in df['Tweets']]

    twitter_streamer = Twitterstreamer()
    twitter_streamer.stream_tweets(fetched_tweet_filename, hash_tag_list)
    #print(dir(tweets))  gives a list of all information that can be extracted



In [ ]:
#Plotting
t_likes = pd.Series(data=df['likes'].values, index = df['date'])
t_likes.plot(figsize= (164), color='b', label = 'Likes')
rt_likes = pd.Series(data=df['retweets'].values, index = df['date'])
rt_likes.plot(figsize= (164), color='r', label = 'Retweets')
plt.xlabel('Tweet Created on')
plt.ylabel('Tweet likes')
plt.legend()
plt.show()

In [87]:
file = 'C:/Users/laksh/tweets.json'
twitter_data = []
with open(file) as f:
    f  = f.readlines()
for line in f:
    twitter_data.append(line)
twitter_data = list(filter(lambda a: a != '\n', twitter_data))

In [103]:
twitter_data[1000]

'{"limit":{"track":29,"timestamp_ms":"1561583347077"}}\n'

In [105]:
loaded_json = []
j=0
for i in twitter_data:
    try:
        j=j+1
        loaded_json.append(' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," " ,json.loads(i)['text']).split()))
    except:
        pass

In [106]:
len(loaded_json)

22147